In [23]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from random import sample
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from itertools import chain
nltk.download('words')
%pylab inline

df = pd.read_json('yelp_academic_dataset_review_cleaned.json', lines=True)
df.head()

Populating the interactive namespace from numpy and matplotlib


C:\Users\tsblo\Anaconda2\envs\p36workshop\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['sample']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1,Q1sbwvVQXV2734tPgoKj4Q,1,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA
1,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0,GJXCdrto3ASJOqKeVWPi6Q,5,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg
2,WTqjgwHlXbSFevF32_DJVw,0,2016-11-09 20:09:03,0,2TzJjDVDEuAW6MR5Vuc1ug,5,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw
3,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0,yi0R0Ugj_xUx_Nek0-_Qig,5,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg
4,b1b1eb3uo-w561D0ZfCEiQ,0,2018-01-30 23:07:38,0,11a8sVPMUFtaC7_ABRkmtw,1,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ


In [24]:
df_full.shape

(102292, 9)

In [27]:
df = df.sample(frac=0.1, replace=False, random_state=1)
df.shape

(10229, 9)

In [28]:
%%time
# tokenization = split sentences into word strings
df['text_list'] = df['text'].map(nltk.word_tokenize)
df['tokens'] = df['text_list'].map(lambda x: set(x)) #set is like a list but removes duplicates

Wall time: 22.2 s


In [30]:
%%time
# lemmatization = converting a word to its base form, different from stemming
lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df['tokens'].map(lambda x: [lemmatizer.lemmatize(word) for word in x])
df['lemmatized'] = df['lemmatized'].map(lambda x: set(x))

# remove stop words
stop_words = set(stopwords.words('english'))
df['processed'] = df['lemmatized'].map(lambda x: [word for word in x if word.lower() not in stop_words])

# remove punctuations
punc = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~..."
df['processed'] = df['processed'].map(lambda x: [word for word in x if word.lower() not in punc])

# remove some other stuff and return lower case
others = ["''", "``", "n't", "l", "'m", "'s"]
df['processed'] = df['processed'].map(lambda x: [word.lower() for word in x if word.lower() not in others])

df.head()

Wall time: 13.4 s


,business_id,cool,date,funny,review_id,stars,text,useful,user_id,text_list,tokens,lemmatized,processed
29369,kPR47uxGqmQNIQkIZr2_Tg,0,2016-06-12 04:28:37,0,kJa4M0WsQaGwoMDSP0c-hw,5,Friendly staff and generous portions of great ...,1,sizanx4RIwRC3BfhQv8KOg,"[Friendly, staff, and, generous, portions, of,...","{generous, staff, and, food, !, u, Friday, of,...","{generous, food, staff, !, and, rock, u, Frida...","[generous, food, staff, rock, u, friday, frien..."
80459,qHQPvp6pZ75fB63kOKUPqg,0,2014-08-13 19:10:43,1,8qst2v9HE8tV7B3IFvtSIA,5,Job well done!! Thank you Rick for making our ...,2,DXP3K7wG_nu9jYPzQ8xVFA,"[Job, well, done, !, !, Thank, you, Rick, for,...","{the, for, and, look, a, our, ., on, much, was...","{the, for, and, look, a, our, ., on, yearly, c...","[look, yearly, carpet, sure, thank, clean, bes..."
21659,vhIJ91MDgUuk4Cr9Kpj1Nw,0,2012-04-29 10:58:50,0,DYAIyC476YkouqJgXuUv_A,4,I'm a big fan of Jimmy John's. This location i...,1,OTRxfMCtfrbUemIigxCaOg,"[I, 'm, a, big, fan, of, Jimmy, John, 's, ., T...","{and, get, unfriendly, bacon, on, service, loc...","{and, get, unfriendly, bacon, on, service, loc...","[get, unfriendly, bacon, service, location, mi..."
69369,oH4iqq4kjJfXpHCgB9G1sw,0,2013-11-14 21:10:14,0,vCJS9ok_G3OfuxCtT3UNYg,4,Well Spa was surprisingly posh and I loved it!...,0,d84GXn6uhT1qyDEziAmR8Q,"[Well, Spa, was, surprisingly, posh, and, I, l...","{the, and, massage, full, posh, loved, ., it, ...","{the, and, massage, amenity, full, posh, loved...","[massage, amenity, full, posh, loved, great, p..."
93624,kPR47uxGqmQNIQkIZr2_Tg,0,2015-07-17 22:52:49,0,rMpvc3Hua7qRtDph8_pnpw,5,Wanted to try this place for a while and it di...,0,Ixaz7mLwzGQGg_W2yfBGqg,"[Wanted, to, try, this, place, for, a, while, ...","{for, and, food, banana, a, ordering, -, our, ...","{for, food, banana, and, dessert, a, ordering,...","[food, banana, dessert, ordering, help, delici..."


In [39]:
vocab_list = []

def extract_words(text):
    for word in text:
        vocab_list.append(word)

In [47]:
df["processed"].map(extract_words)

29369     None
80459     None
21659     None
69369     None
93624     None
2576      None
79062     None
34064     None
12698     None
101325    None
64011     None
91350     None
69334     None
54226     None
33096     None
95739     None
88453     None
26324     None
17801     None
27883     None
96503     None
17910     None
67133     None
14981     None
65232     None
41526     None
99308     None
6087      None
52792     None
86427     None
          ... 
13650     None
60924     None
80832     None
55219     None
42542     None
80386     None
18198     None
30170     None
378       None
97323     None
93812     None
39        None
85022     None
99682     None
15459     None
57417     None
1298      None
76795     None
24395     None
61577     None
31194     None
44875     None
101962    None
10084     None
91058     None
14409     None
101424    None
15957     None
38912     None
85374     None
Name: processed, Length: 10229, dtype: object

In [48]:
vocab_list = list(set(vocab_list))

In [49]:
len(vocab_list)

30284

In [56]:
from nltk.corpus import words

empty_list = []

for each in vocab_list:
    if each in words.words():
        empty_list.append(each)

In [57]:
len(empty_list)

12263

In [75]:
from collections import Counter

L = empty_list

f = lambda x: Counter([y for y in x if y in L])

df['bag_vector'] = (pd.DataFrame(df['processed'].apply(f).values.tolist())
               .fillna(0)
               .astype(int)
               .reindex(columns=L)
               .values
               .tolist())

In [76]:
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,text_list,tokens,lemmatized,processed,bag_vector
29369,kPR47uxGqmQNIQkIZr2_Tg,0,2016-06-12 04:28:37,0,kJa4M0WsQaGwoMDSP0c-hw,5,Friendly staff and generous portions of great ...,1,sizanx4RIwRC3BfhQv8KOg,"[Friendly, staff, and, generous, portions, of,...","{generous, staff, and, food, !, u, Friday, of,...","{generous, food, staff, !, and, rock, u, Frida...","[generous, food, staff, rock, u, friday, frien...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
80459,qHQPvp6pZ75fB63kOKUPqg,0,2014-08-13 19:10:43,1,8qst2v9HE8tV7B3IFvtSIA,5,Job well done!! Thank you Rick for making our ...,2,DXP3K7wG_nu9jYPzQ8xVFA,"[Job, well, done, !, !, Thank, you, Rick, for,...","{the, for, and, look, a, our, ., on, much, was...","{the, for, and, look, a, our, ., on, yearly, c...","[look, yearly, carpet, sure, thank, clean, bes...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
21659,vhIJ91MDgUuk4Cr9Kpj1Nw,0,2012-04-29 10:58:50,0,DYAIyC476YkouqJgXuUv_A,4,I'm a big fan of Jimmy John's. This location i...,1,OTRxfMCtfrbUemIigxCaOg,"[I, 'm, a, big, fan, of, Jimmy, John, 's, ., T...","{and, get, unfriendly, bacon, on, service, loc...","{and, get, unfriendly, bacon, on, service, loc...","[get, unfriendly, bacon, service, location, mi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
69369,oH4iqq4kjJfXpHCgB9G1sw,0,2013-11-14 21:10:14,0,vCJS9ok_G3OfuxCtT3UNYg,4,Well Spa was surprisingly posh and I loved it!...,0,d84GXn6uhT1qyDEziAmR8Q,"[Well, Spa, was, surprisingly, posh, and, I, l...","{the, and, massage, full, posh, loved, ., it, ...","{the, and, massage, amenity, full, posh, loved...","[massage, amenity, full, posh, loved, great, p...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
93624,kPR47uxGqmQNIQkIZr2_Tg,0,2015-07-17 22:52:49,0,rMpvc3Hua7qRtDph8_pnpw,5,Wanted to try this place for a while and it di...,0,Ixaz7mLwzGQGg_W2yfBGqg,"[Wanted, to, try, this, place, for, a, while, ...","{for, and, food, banana, a, ordering, -, our, ...","{for, food, banana, and, dessert, a, ordering,...","[food, banana, dessert, ordering, help, delici...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [84]:
#sanity check
def Average(lst): 
    return sum(lst) / len(lst)

first_vect = df["bag_vector"].iloc[0]
Average(first_vect)

0.0008154611432765229

In [86]:
vocab_df = pd.DataFrame(L) 
vocab_df.shape

(12263, 1)

In [87]:
df.to_csv("bag_of_words.csv")
vocab_df.to_csv("vocab_df.csv")